In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from tqdm import tqdm
from typing import Tuple, List, Optional, Dict, Any, Union, Callable

def simulate_buffon_needle(
    num_needles: int, 
    needle_length: float = 1, 
    line_distance: float = 1, 
    floor_width: float = 10
) -> Tuple[float, int, List[Tuple[float, float, float]]]:
    """
    Simulate Buffon's needle experiment.
    
    Parameters:
    -----------
    num_needles : int
        Number of needles to drop
    needle_length : float
        Length of each needle (should be <= line_distance)
    line_distance : float
        Distance between parallel lines
    floor_width : float
        Total width of the floor for visualization
    
    Returns:
    --------
    Tuple[float, int, List[Tuple[float, float, float]]]
        (estimated_pi, crossings, needles)
        where needles is a list of (x, y, theta) for each needle
    """
    # validate needle length
    if needle_length > line_distance:
        raise ValueError("Needle length must be less than or equal to line distance")
    
    # random positions and angles for needles
    # for visualization: x positions across the entire floor
    x_positions = np.random.uniform(0, floor_width, num_needles)
    thetas = np.random.uniform(0, np.pi, num_needles)
    
    # calculate y positions (0-10 for visualization)
    y_positions = np.random.uniform(0, 10, num_needles)
    
    # for calculation: find distance to nearest line for each needle
    # lines are at positions 0, line_distance, 2*line_distance, etc.
    nearest_line_positions = np.floor(x_positions / line_distance) * line_distance
    distances_to_nearest_line = x_positions - nearest_line_positions
    
    # determine which needles cross a line
    needle_half_length = needle_length / 2
    
    # a needle crosses a line if distance to nearest line < (L/2)*sin(theta)
    # or if distance to next line > line_distance - (L/2)*sin(theta)
    crossing_left = distances_to_nearest_line < needle_half_length * np.sin(thetas)
    crossing_right = distances_to_nearest_line > line_distance - needle_half_length * np.sin(thetas)
    crossings = np.sum(crossing_left | crossing_right)
    
    # calculate estimated π
    if crossings > 0:
        estimated_pi = (2 * needle_length * num_needles) / (crossings * line_distance)
    else:
        estimated_pi = float('inf')
    
    # prepare needle data for visualization
    needles = [(x_positions[i], y_positions[i], thetas[i]) for i in range(num_needles)]
    
    return estimated_pi, crossings, needles

def plot_experiment(
    needles: List[Tuple[float, float, float]], 
    line_distance: float = 1, 
    max_y: float = 10, 
    needle_length: float = 1, 
    highlight_crossings: bool = True
) -> plt.Figure:
    """
    Visualize the Buffon's needle experiment.
    
    Parameters:
    -----------
    needles : List[Tuple[float, float, float]]
        List of (x, y, theta) for each needle
    line_distance : float
        Distance between parallel lines
    max_y : float
        Maximum y value for visualization
    needle_length : float
        Length of each needle
    highlight_crossings : bool
        Whether to highlight needles that cross lines
        
    Returns:
    --------
    plt.Figure
        The matplotlib figure object
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # draw parallel lines
    num_lines = int(10 / line_distance) + 1
    for i in range(num_lines):
        ax.axvline(x=i*line_distance, color='black', linestyle='-', alpha=0.7)
    
    # half-length of the needle
    half_length = needle_length / 2
    
    # draw needles
    crossing_needles = []
    non_crossing_needles = []
    
    for x, y, theta in needles:
        # calculate needle endpoints
        dx = half_length * np.sin(theta)
        dy = half_length * np.cos(theta)
        
        x1, y1 = x - dx, y - dy
        x2, y2 = x + dx, y + dy
        
        # check if needle crosses a line
        crosses = False
        line_positions = [i*line_distance for i in range(num_lines)]
        for line_pos in line_positions:
            # if the needle endpoints are on opposite sides of the line
            if (x1 - line_pos) * (x2 - line_pos) <= 0:
                crosses = True
                break
        
        if crosses:
            crossing_needles.append([(x1, y1), (x2, y2)])
        else:
            non_crossing_needles.append([(x1, y1), (x2, y2)])
    
    # plot non-crossing needles
    for needle in non_crossing_needles:
        (x1, y1), (x2, y2) = needle
        ax.plot([x1, x2], [y1, y2], 'b-', alpha=0.5, linewidth=1)
    
    # plot crossing needles
    for needle in crossing_needles:
        (x1, y1), (x2, y2) = needle
        ax.plot([x1, x2], [y1, y2], 'r-', alpha=0.7, linewidth=1.5)
    
    # add legend
    legend_elements = [
        Line2D([0], [0], color='blue', lw=1.5, alpha=0.5, label='Non-crossing needles'),
        Line2D([0], [0], color='red', lw=1.5, alpha=0.7, label='Crossing needles'),
        Line2D([0], [0], color='black', lw=1.5, alpha=0.7, label='Lines')
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    # set axis limits and labels
    ax.set_xlim(-0.5, (num_lines-1)*line_distance + 0.5)
    ax.set_ylim(-0.5, max_y + 0.5)
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    ax.set_title("Buffon's Needle Experiment Visualization")
    
    return fig

def run_buffon_experiment(
    max_needles: int = 10000, 
    step: int = 1000, 
    needle_length: float = 1, 
    line_distance: float = 1, 
    floor_width: float = 10
) -> Tuple[range, List[float]]:
    """
    Run Buffon's needle experiment with increasing numbers of needles.
    
    Parameters:
    -----------
    max_needles : int
        Maximum number of needles to simulate
    step : int
        Step size for increasing number of needles
    needle_length : float
        Length of each needle
    line_distance : float
        Distance between parallel lines
    floor_width : float
        Width of the entire floor for visualization
        
    Returns:
    --------
    Tuple[range, List[float]]
        (needle_counts, pi_estimates) where needle_counts is a range object
        and pi_estimates is a list of estimated π values
    """
    needle_counts = range(step, max_needles + step, step)
    pi_estimates = []
    errors = []
    
    for count in tqdm(needle_counts, desc="Simulating needle drops"):
        estimated_pi, _, _ = simulate_buffon_needle(count, needle_length, line_distance, floor_width)
        pi_estimates.append(estimated_pi)
        errors.append(abs(estimated_pi - np.pi)/np.pi * 100)  # calculate percentage error
    
    # plot results
    fig, ax1 = plt.subplots(1, 1, figsize=(12, 10), sharex=True)
    
    # plot 1: estimated π values
    ax1.plot(needle_counts, pi_estimates, 'b-', alpha=0.7)
    ax1.axhline(y=np.pi, color='r', linestyle='--', alpha=0.7, label=f'π = {np.pi:.6f}')
    ax1.set_ylabel('Estimated π')
    ax1.set_title('Estimated Value of π vs. Number of Needles')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    plt.tight_layout()
    
    return needle_counts, pi_estimates

# example usage
if __name__ == "__main__":
    np.random.seed(42)  # for reproducibility
    
    # simulation parameters
    needle_length = 0.8
    line_distance = 1.0
    floor_width = 10.0  # width of the entire floor
    
    # small visualization example
    num_needles_viz = 10000
    estimated_pi, crossings, needles = simulate_buffon_needle(
        num_needles_viz, 
        needle_length, 
        line_distance, 
        floor_width
    )
    print(f"Dropped {num_needles_viz} needles, {crossings} crossed lines")
    print(f"Estimated π = {estimated_pi:.6f} (Actual π = {np.pi:.6f})")
    
    # visualize the experiment
    fig = plot_experiment(needles, line_distance, 10, needle_length)
    plt.tight_layout()
    plt.savefig('buffon_needle_visualization.png', dpi=300)
    
    # run a larger experiment to see convergence
    needle_counts, pi_estimates = run_buffon_experiment(
        max_needles=100000, 
        step=1000, 
        needle_length=needle_length, 
        line_distance=line_distance
    )
    
    # print final estimate
    final_estimate = pi_estimates[-1]
    print(f"\nFinal estimate with {needle_counts[-1]} needles: π ≈ {final_estimate:.6f}")
    print(f"Error: {abs(final_estimate - np.pi):.6f} ({abs(final_estimate - np.pi)/np.pi*100:.4f}%)")
    
    plt.savefig('buffon_pi_convergence.png', dpi=300)
    plt.show()